#### Import Modules

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os, glob

from keras import models
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, Deconvolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.utils.visualize_util import plot

from keras.callbacks import *
from keras import backend as K

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


#### Set Paths and Load Dataset

In [2]:
train_input_path = './data/CamVid/train_input.npy'
train_label_path = './data/CamVid/train_label.npy'
test_input_path = './data/CamVid/test_input.npy'
test_label_path = './data/CamVid/test_label.npy'

train_input_mat = np.load(train_input_path)
train_label_mat = np.load(train_label_path)
test_input_mat = np.load(test_input_path)
test_label_mat = np.load(test_label_path)

n_train = train_input_mat.shape[0]
n_test = test_input_mat.shape[0]
img_width = train_input_mat.shape[1]
img_height = train_input_mat.shape[2]
n_class = 12

train_label_mat_bin = np.reshape(to_categorical(np.reshape(train_label_mat, -1)), 
                                 np.shape(train_label_mat) +(n_class,))
test_label_mat_bin = np.reshape(to_categorical(np.reshape(test_label_mat, -1)), 
                                np.shape(test_label_mat) +(n_class,))

print train_input_mat.shape, train_label_mat.shape, train_label_mat_bin.shape
print test_input_mat.shape, test_label_mat.shape, test_label_mat_bin.shape

(530, 360, 480, 3) (530, 360, 480) (530, 360, 480, 12)
(171, 360, 480, 3) (171, 360, 480) (171, 360, 480, 12)


#### Define Model

In [3]:
def SegNet():

    # Encoder Part ===================================================================
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(img_width, img_height, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())

    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D())

    # Bottleneck Part ===================================================================
    model.add(Convolution2D(256, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # Decoder Part
    model.add(UpSampling2D())
    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(UpSampling2D())
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(UpSampling2D())
    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(n_class, 3, 3, border_mode='same'))
    model.add(BatchNormalization())

    # Reshape for Cross-Entropy Calculation
    model.add(Reshape((img_width*img_height, n_class)))
    model.add(Activation('softmax'))
    return model

#### Compile the Model

In [ ]:
seg_model = SegNet()
optimizer = SGD(lr=0.001, momentum=0.9, decay=0.0005, nesterov=False)
seg_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
print 'Model Compile Done!'

Model Compile Done!


#### Start Training

In [ ]:
print "Starting Training..."

train_X = train_input_mat
train_Y = np.reshape(train_label_mat_bin, (n_train, img_width*img_height, n_class))
test_X = test_input_mat
test_Y = np.reshape(test_label_mat_bin, (n_test, img_width*img_height, n_class))

checkpoint_path="./checkpoint/CamVid_epoch_{epoch:02d}_acc_{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=0, 
                             save_best_only=True, save_weights_only=False, mode='auto')
seg_model.fit(train_X, train_Y, nb_epoch=20, batch_size=16, verbose=1, validation_data=(test_X, test_Y), callbacks=[checkpoint])

print "Training Completed!!"

Starting Training...
Train on 530 samples, validate on 171 samples
Epoch 1/20
530/530 [==============================] - 77s - loss: 2.1793 - acc: 0.3236 - val_loss: 2.3919 - val_acc: 0.1973
Epoch 2/20
530/530 [==============================] - 61s - loss: 1.6147 - acc: 0.5647 - val_loss: 2.1971 - val_acc: 0.3229
Epoch 3/20
530/530 [==============================] - 62s - loss: 1.4242 - acc: 0.6534 - val_loss: 2.0207 - val_acc: 0.4187
Epoch 4/20
530/530 [==============================] - 63s - loss: 1.3100 - acc: 0.6966 - val_loss: 1.8397 - val_acc: 0.4940
Epoch 5/20
530/530 [==============================] - 63s - loss: 1.2169 - acc: 0.7229 - val_loss: 1.6827 - val_acc: 0.5181
Epoch 6/20
530/530 [==============================] - 63s - loss: 1.1403 - acc: 0.7445 - val_loss: 1.5465 - val_acc: 0.5732
Epoch 7/20
530/530 [==============================] - 63s - loss: 1.0793 - acc: 0.7582 - val_loss: 1.4573 - val_acc: 0.6130
Epoch 8/20
530/530 [==============================] - 63s - loss:

#### Segment Test Scenes

In [ ]:
predicted = model.predict(test_X)
predicted = np.reshape(predicted, (n_test, img_width, img_height, n_class))
print np.shape(predicted)

#### Validate Segmentation Results

In [ ]:
vis_ind = 100
label_img = np.argmax(test_label_mat_bin[vis_ind, :, :, :], axis=2)
pred_img = np.argmax(predicted[vis_ind, :, :, :], axis=2)

plt.imshow(test_X[vis_ind, :, :, :])

plt.figure()
plt.imshow(label_img)

plt.figure()
plt.imshow(pred_img)